In [16]:
import pandas as pd
import re
import os
from dotenv import load_dotenv 

In [17]:
load_dotenv()

False

In [18]:
# --- 1. Dummy Data Loading (Replace with your actual DLD and rental data loading) ---
# Assuming your DLD data has columns like:
# 'property_id', 'area', 'property_type', 'sale_price', 'transaction_date', 'area_sqft'
# Assuming your rental data has:
# 'property_id', 'area', 'property_type', 'annual_rent'

# Create dummy dataframes for demonstration
dld_data = pd.DataFrame({
    'property_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'area': ['Business Bay', 'Business Bay', 'JVC', 'JVC', 'Dubai Marina',
             'Downtown Dubai', 'Business Bay', 'JVC', 'Dubai Marina', 'Downtown Dubai'],
    'property_type': ['Studio', '1BR', 'Studio', '2BR', '1BR',
                      'Studio', 'Studio', '1BR', 'Studio', '2BR'],
    'sale_price': [950000, 1500000, 600000, 1100000, 1800000,
                   1200000, 920000, 850000, 1400000, 2200000],
    'transaction_date': pd.to_datetime(['2024-01-15', '2024-02-20', '2024-03-10', '2024-04-05', '2024-05-01',
                                        '2024-06-12', '2024-07-01', '2024-08-15', '2024-09-01', '2024-10-10']),
    'area_sqft': [400, 700, 350, 900, 800,
                  420, 390, 650, 410, 1200]
})

rental_data = pd.DataFrame({
    'property_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], # Assuming same property IDs for simplicity
    'area': ['Business Bay', 'Business Bay', 'JVC', 'JVC', 'Dubai Marina',
             'Downtown Dubai', 'Business Bay', 'JVC', 'Dubai Marina', 'Downtown Dubai'],
    'property_type': ['Studio', '1BR', 'Studio', '2BR', '1BR',
                      'Studio', 'Studio', '1BR', 'Studio', '2BR'],
    'annual_rent': [70000, 100000, 50000, 85000, 95000,
                    70000, 68000, 75000, 88000, 130000]
})


In [19]:
dld_data

,property_id,area,property_type,sale_price,transaction_date,area_sqft
0,1,Business Bay,Studio,950000,2024-01-15,400
1,2,Business Bay,1BR,1500000,2024-02-20,700
2,3,JVC,Studio,600000,2024-03-10,350
3,4,JVC,2BR,1100000,2024-04-05,900
4,5,Dubai Marina,1BR,1800000,2024-05-01,800
5,6,Downtown Dubai,Studio,1200000,2024-06-12,420
6,7,Business Bay,Studio,920000,2024-07-01,390
7,8,JVC,1BR,850000,2024-08-15,650
8,9,Dubai Marina,Studio,1400000,2024-09-01,410
9,10,Downtown Dubai,2BR,2200000,2024-10-10,1200


In [20]:
rental_data

,property_id,area,property_type,annual_rent
0,1,Business Bay,Studio,70000
1,2,Business Bay,1BR,100000
2,3,JVC,Studio,50000
3,4,JVC,2BR,85000
4,5,Dubai Marina,1BR,95000
5,6,Downtown Dubai,Studio,70000
6,7,Business Bay,Studio,68000
7,8,JVC,1BR,75000
8,9,Dubai Marina,Studio,88000
9,10,Downtown Dubai,2BR,130000


In [21]:
# --- 2. Data Cleaning & Feature Engineering ---

# Merge data (basic merge, you might need more complex logic if IDs don't align perfectly)
merged_data = pd.merge(dld_data, rental_data, on=['property_id', 'area', 'property_type'], how='left')

# Calculate ROI (Gross Rental Yield)
# Note: For precise ROI, you'd need net income after expenses, but for simplicity, we use gross.
merged_data['roi_rental'] = (merged_data['annual_rent'] / merged_data['sale_price']) * 100

# Calculate Price per Square Foot (PSF)
merged_data['price_per_sqft'] = merged_data['sale_price'] / merged_data['area_sqft']


In [22]:
merged_data

,property_id,area,property_type,sale_price,transaction_date,area_sqft,annual_rent,roi_rental,price_per_sqft
0,1,Business Bay,Studio,950000,2024-01-15,400,70000,7.368421,2375.000000
1,2,Business Bay,1BR,1500000,2024-02-20,700,100000,6.666667,2142.857143
2,3,JVC,Studio,600000,2024-03-10,350,50000,8.333333,1714.285714
3,4,JVC,2BR,1100000,2024-04-05,900,85000,7.727273,1222.222222
4,5,Dubai Marina,1BR,1800000,2024-05-01,800,95000,5.277778,2250.000000
5,6,Downtown Dubai,Studio,1200000,2024-06-12,420,70000,5.833333,2857.142857
6,7,Business Bay,Studio,920000,2024-07-01,390,68000,7.391304,2358.974359
7,8,JVC,1BR,850000,2024-08-15,650,75000,8.823529,1307.692308
8,9,Dubai Marina,Studio,1400000,2024-09-01,410,88000,6.285714,3414.634146
9,10,Downtown Dubai,2BR,2200000,2024-10-10,1200,130000,5.909091,1833.333333


In [23]:
# Aggregate data for quick lookups and ranges (e.g., for Business Bay Studio)
# This will be your "knowledge base" for the engine
summary_data = merged_data.groupby(['area', 'property_type']).agg(
    avg_sale_price=('sale_price', 'mean'),
    min_sale_price=('sale_price', 'min'),
    max_sale_price=('sale_price', 'max'),
    avg_roi_rental=('roi_rental', 'mean'),
    count=('property_id', 'count')
).reset_index()
# Round ROI for better readability
summary_data['avg_roi_rental'] = summary_data['avg_roi_rental'].round(2)
summary_data['avg_sale_price'] = summary_data['avg_sale_price'].round(0)
summary_data['min_sale_price'] = summary_data['min_sale_price'].round(0)
summary_data['max_sale_price'] = summary_data['max_sale_price'].round(0)

In [24]:
summary_data

,area,property_type,avg_sale_price,min_sale_price,max_sale_price,avg_roi_rental,count
0,Business Bay,1BR,1500000.0,1500000,1500000,6.67,1
1,Business Bay,Studio,935000.0,920000,950000,7.38,2
2,Downtown Dubai,2BR,2200000.0,2200000,2200000,5.91,1
3,Downtown Dubai,Studio,1200000.0,1200000,1200000,5.83,1
4,Dubai Marina,1BR,1800000.0,1800000,1800000,5.28,1
5,Dubai Marina,Studio,1400000.0,1400000,1400000,6.29,1
6,JVC,1BR,850000.0,850000,850000,8.82,1
7,JVC,2BR,1100000.0,1100000,1100000,7.73,1
8,JVC,Studio,600000.0,600000,600000,8.33,1


In [25]:
# --- 3. Recommendation Engine Logic ---

def parse_query(query):
    """
    Parses a user query to extract cost, ROI, and location.
    Basic regex parsing for demonstration. More robust NLP might be needed.
    """
    cost_match = re.search(r'(\d+K|\d+\s*million|\d+\s*M|\d+)', query, re.IGNORECASE)
    roi_match = re.search(r'roi (above|over|more than)?\s*(\d+\.?\d*)%?', query, re.IGNORECASE)
    location_match = re.search(r'(business bay|jvc|dubai marina|downtown dubai|international city)', query, re.IGNORECASE)
    property_type_match = re.search(r'(studio|1br|2br|villa|townhouse)', query, re.IGNORECASE)

    parsed_cost = None
    if cost_match:
        cost_str = cost_match.group(0).lower().replace(' ', '')
        if 'k' in cost_str:
            parsed_cost = float(cost_str.replace('k', '')) * 1000
        elif 'million' in cost_str or 'm' in cost_str:
            parsed_cost = float(cost_str.replace('million', '').replace('m', '')) * 1000000
        else:
            parsed_cost = float(cost_str) # Assume it's in AED directly

    parsed_roi = None
    if roi_match:
        parsed_roi = float(roi_match.group(2)) # Get the percentage value

    parsed_location = location_match.group(0).title() if location_match else None
    parsed_property_type = property_type_match.group(0).title() if property_type_match else None

    return {
        'cost': parsed_cost,
        'roi': parsed_roi,
        'location': parsed_location,
        'property_type': parsed_property_type
    }

def get_recommendations(query_params, data):
    """
    Provides property recommendations based on parsed query parameters.
    """
    filtered_data = data.copy()

    if query_params['location']:
        filtered_data = filtered_data[filtered_data['area'].str.contains(query_params['location'], case=False, na=False)]
        if filtered_data.empty:
            return f"Sorry, I couldn't find data for {query_params['location']} that matches your criteria."

    if query_params['property_type']:
        filtered_data = filtered_data[filtered_data['property_type'].str.contains(query_params['property_type'], case=False, na=False)]
        if filtered_data.empty:
            return "No properties of that type found in the selected location."

    if query_params['roi']:
        # Filter based on average ROI for the specific area/property type combination
        filtered_data = filtered_data[filtered_data['avg_roi_rental'] >= query_params['roi']]
        if filtered_data.empty:
            return "No properties found with that ROI in the selected criteria."

    if query_params['cost']:
        # Filter by properties where average price is within a certain range of the target cost
        # For a simple recommendation, let's say +/- 20% of the desired cost
        min_cost = query_params['cost'] * 0.5
        max_cost = query_params['cost'] * 1.2
        filtered_data = filtered_data[
            (filtered_data['avg_sale_price'] >= min_cost) &
            (filtered_data['avg_sale_price'] <= max_cost)
        ]
        if filtered_data.empty:
            return "No properties found within that cost range for the selected criteria."

    if filtered_data.empty:
        return "No recommendations found for your specific criteria. Please try a broader search."

    # Sort by ROI descending and take top N
    filtered_data = filtered_data.sort_values(by='avg_roi_rental', ascending=False)

    recommendations = []
    for _, row in filtered_data.iterrows():
        rec = (
            f"Location: {row['area']}, Property Type: {row['property_type']}\n"
            f"  Estimated Sale Price: AED {int(row['min_sale_price']):,} - {int(row['max_sale_price']):,}\n"
            f"  Average Rental ROI: {row['avg_roi_rental']}%"
        )
        recommendations.append(rec)

    return "\n---\n".join(recommendations[:5]) # Return top 5 recommendations

In [26]:
print("Welcome to the Dubai Property Recommendation Engine!")
print("You can ask about cost, desired ROI, and location (e.g., 'Studio in Business Bay around 950K with ROI above 7%').")
print("Type 'exit' to quit.")

Welcome to the Dubai Property Recommendation Engine!
You can ask about cost, desired ROI, and location (e.g., 'Studio in Business Bay around 950K with ROI above 7%').
Type 'exit' to quit.


In [27]:
query = 'Studio around 840K with ROI above 8%'
params = parse_query(query)
print(params)
result = get_recommendations(params, summary_data)
print("\n--- Recommendations ---")
print(result)

{'cost': 840000.0, 'roi': 8.0, 'location': None, 'property_type': 'Studio'}

--- Recommendations ---
Location: JVC, Property Type: Studio
  Estimated Sale Price: AED 600,000 - 600,000
  Average Rental ROI: 8.33%
